# Home and Work Assign RID

This code is used to re-assign the roadID for each workplace. The distance function in geopandas fialed to generate the correct distance, so I create a new distance function in this code

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import timeit
import multiprocessing
from math import sin, cos, sqrt, atan2, radians

### Load Data

In [2]:
#Home CT
homect = gpd.read_file('../Shp_Study_Area_Richard/Home_Point/Home_By_State/home_ct.shp')
print("CT size", len(homect))
#Home NJ
#Home PA
#homepa = gpd.read_file('../Shp_Study_Area_Richard/Home_Point/Home_By_State/home_pa.shp')
#print("PA size", len(homepa))

CT size 1326196


In [3]:
homect.head()

,hhold,long,lat,geometry
0,09001010101h0,-73.698597,41.112328,POINT (-73.69860 41.11233)
1,09001010101h1,-73.670429,41.097875,POINT (-73.67043 41.09788)
2,09001010101h10,-73.667156,41.046854,POINT (-73.66716 41.04685)
3,09001010101h100,-73.720042,41.099381,POINT (-73.72004 41.09938)
4,09001010101h1000,-73.684385,41.056047,POINT (-73.68439 41.05605)


In [4]:
#Road
road = gpd.read_file('../Shp_Study_Area_Richard/Road_Point/road_points_Aug_Richard.shp')
road.shape

(1250082, 5)

In [7]:
road['RID'] = road['RDID']

In [8]:
road.head()

,RDID,STATE,Long,Lat,geometry,RID
0,1,34,-74.658902,40.157278,POINT (-74.65890 40.15728),1
1,2,09,-73.227316,41.225055,POINT (-73.22732 41.22506),2
2,3,09,-73.225156,41.225181,POINT (-73.22516 41.22518),3
3,4,09,-73.221523,41.232643,POINT (-73.22152 41.23264),4
4,5,09,-73.210993,41.235990,POINT (-73.21099 41.23599),5


## Functions

In [9]:
def new_distance(x1, y1, x2, y2):
    # approximate radius of earth in km
    R = 6373.0
    
    lat1 = radians (x1)
    long1 = radians (y1)
    lat2 = radians(x2)
    long2 = radians(y2)
    
    dlon = long2 - long1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [10]:
def assign_rid(data, road):
    #Geo Location
    d_geometry = data.geometry
    #print(d_geometry)
    
    #Create Buffer for each Home or Work Point
    buff = data.geometry.buffer(0.001)
    # Lat and Long
    x = data.lat
    y = data.long
    #print("Site Lat",x)
    #print("Site Long", y)
    
    #Find Intersected points
    r_in = road[road.intersects(buff)].copy()
    #print("Intersected road points numbers:", len(r_in))

    if r_in.empty:
        #print(data.wrkID, "First Empty")
        buff = data.geometry.buffer(0.02)
        r_in = road[road.intersects(buff)].copy()
        #Intersect Road Point Lat list
        rx = r_in.loc[:,'Lat'].tolist()
        #Intersect Road Point Long list
        ry = r_in.loc[:, 'Long'].tolist()
        
        #Calculate Distance between the point and intersected road points 
        dist = []
        for j in range(0, len(r_in)):
            d = new_distance(x, y, rx[j], ry[j])
            dist.append(d)  
        #print("Distance list's length",len(dist))
            
        rid = r_in.loc[:,'RDID']
        #Create DF
        df_rd = pd.DataFrame({'RDID': rid, 'Dist':dist}).sort_values(by='Dist')
        #print(df_rd.head(1))
        #print(df_rd.iloc[0, 0])
        #r_id.append(df_rd.iloc[0, 0])
        return df_rd.iloc[0, 0]
    
    else:
        #Intersect Road Point Lat list
        rx = r_in.loc[:,'Lat'].tolist()
        #Intersect Road Point Long list
        ry = r_in.loc[:, 'Long'].tolist()
        
        #Calculate Distance between the point and intersected road points 
        dist = []
        for j in range(0, len(r_in)):
            d = new_distance(x, y, rx[j], ry[j])
            dist.append(d)  
        #print("Distance list's length",len(dist))
            
        rid = r_in.loc[:,'RDID']
        #Create DF
        df_rd = pd.DataFrame({'RDID': rid, 'Dist':dist}).sort_values(by='Dist')
        #print(df_rd.head(1))
        #print(df_rd.iloc[0, 0])
        #r_id.append(df_rd.iloc[0, 0])
        return df_rd.iloc[0, 0]

In [11]:
def go(data):
    data['RID'] = data.apply(assign_rid, args=(road,),axis=1)
    return data
    
def parallelize(data, func):
    #Cores
    num_cores = multiprocessing.cpu_count()-1
    num_partitions = num_cores
    #Split
    data_split = np.array_split(data, num_partitions)
    #Pools
    pool = multiprocessing.Pool(num_cores)
    
    df = pd.concat(pool.map(func, data_split))
    return df

    pool.close()
    pool.join()

### Assign Function Test

In [12]:
test = homect.iloc[:10,:].copy()
#test['SrdID'] = np.repeat(0,len(test))

In [13]:
test

,hhold,long,lat,geometry
0,09001010101h0,-73.698597,41.112328,POINT (-73.69860 41.11233)
1,09001010101h1,-73.670429,41.097875,POINT (-73.67043 41.09788)
2,09001010101h10,-73.667156,41.046854,POINT (-73.66716 41.04685)
3,09001010101h100,-73.720042,41.099381,POINT (-73.72004 41.09938)
4,09001010101h1000,-73.684385,41.056047,POINT (-73.68439 41.05605)
5,09001010101h1001,-73.688061,41.090949,POINT (-73.68806 41.09095)
6,09001010101h1002,-73.684679,41.055643,POINT (-73.68468 41.05564)
7,09001010101h1003,-73.689619,41.056749,POINT (-73.68962 41.05675)
8,09001010101h1004,-73.659922,41.048102,POINT (-73.65992 41.04810)
9,09001010101h1005,-73.667787,41.037095,POINT (-73.66779 41.03709)


In [14]:
test['RID'] = test.apply(assign_rid, args=(road,),axis=1)

In [16]:
test

,hhold,long,lat,geometry,RID
0,09001010101h0,-73.698597,41.112328,POINT (-73.69860 41.11233),225136
1,09001010101h1,-73.670429,41.097875,POINT (-73.67043 41.09788),4084
2,09001010101h10,-73.667156,41.046854,POINT (-73.66716 41.04685),129288
3,09001010101h100,-73.720042,41.099381,POINT (-73.72004 41.09938),132056
4,09001010101h1000,-73.684385,41.056047,POINT (-73.68439 41.05605),236212
5,09001010101h1001,-73.688061,41.090949,POINT (-73.68806 41.09095),557005
6,09001010101h1002,-73.684679,41.055643,POINT (-73.68468 41.05564),236212
7,09001010101h1003,-73.689619,41.056749,POINT (-73.68962 41.05675),3275
8,09001010101h1004,-73.659922,41.048102,POINT (-73.65992 41.04810),236166
9,09001010101h1005,-73.667787,41.037095,POINT (-73.66779 41.03709),558618


In [17]:
test.merge(road, on='RID', how='inner')

,hhold,long,lat,geometry_x,RID,RDID,STATE,Long,Lat,geometry_y
0,09001010101h0,-73.698597,41.112328,POINT (-73.69860 41.11233),225136,225136,09,-73.697749,41.113425,POINT (-73.69775 41.11342)
1,09001010101h1,-73.670429,41.097875,POINT (-73.67043 41.09788),4084,4084,09,-73.670365,41.097601,POINT (-73.67037 41.09760)
2,09001010101h10,-73.667156,41.046854,POINT (-73.66716 41.04685),129288,129288,09,-73.667526,41.047768,POINT (-73.66753 41.04777)
3,09001010101h100,-73.720042,41.099381,POINT (-73.72004 41.09938),132056,132056,09,-73.720185,41.099778,POINT (-73.72019 41.09978)
4,09001010101h1000,-73.684385,41.056047,POINT (-73.68439 41.05605),236212,236212,09,-73.684882,41.055706,POINT (-73.68488 41.05571)
5,09001010101h1002,-73.684679,41.055643,POINT (-73.68468 41.05564),236212,236212,09,-73.684882,41.055706,POINT (-73.68488 41.05571)
6,09001010101h1001,-73.688061,41.090949,POINT (-73.68806 41.09095),557005,557005,09,-73.687929,41.090138,POINT (-73.68793 41.09014)
7,09001010101h1003,-73.689619,41.056749,POINT (-73.68962 41.05675),3275,3275,09,-73.690297,41.056475,POINT (-73.69030 41.05648)
8,09001010101h1004,-73.659922,41.048102,POINT (-73.65992 41.04810),236166,236166,09,-73.660382,41.047974,POINT (-73.66038 41.04797)
9,09001010101h1005,-73.667787,41.037095,POINT (-73.66779 41.03709),558618,558618,09,-73.667687,41.037297,POINT (-73.66769 41.03730)


### Parallelize Test

In [18]:
test = homect.iloc[:50,:].copy()
#test['SrdID'] = np.repeat(0,len(test))

In [19]:
#Test 
start_time = timeit.default_timer()
test_p = parallelize(test, go)
elapsed = timeit.default_timer() - start_time
print(elapsed)

132.45157498899994


In [20]:
test_p

,hhold,long,lat,geometry,RID
0,09001010101h0,-73.698597,41.112328,POINT (-73.69860 41.11233),225136
1,09001010101h1,-73.670429,41.097875,POINT (-73.67043 41.09788),4084
2,09001010101h10,-73.667156,41.046854,POINT (-73.66716 41.04685),129288
3,09001010101h100,-73.720042,41.099381,POINT (-73.72004 41.09938),132056
4,09001010101h1000,-73.684385,41.056047,POINT (-73.68439 41.05605),236212
5,09001010101h1001,-73.688061,41.090949,POINT (-73.68806 41.09095),557005
6,09001010101h1002,-73.684679,41.055643,POINT (-73.68468 41.05564),236212
7,09001010101h1003,-73.689619,41.056749,POINT (-73.68962 41.05675),3275
8,09001010101h1004,-73.659922,41.048102,POINT (-73.65992 41.04810),236166
9,09001010101h1005,-73.667787,41.037095,POINT (-73.66779 41.03709),558618


In [21]:
test_p.merge(road, on='RID', how='inner')

,hhold,long,lat,geometry_x,RID,RDID,STATE,Long,Lat,geometry_y
0,09001010101h0,-73.698597,41.112328,POINT (-73.69860 41.11233),225136,225136,09,-73.697749,41.113425,POINT (-73.69775 41.11342)
1,09001010101h1,-73.670429,41.097875,POINT (-73.67043 41.09788),4084,4084,09,-73.670365,41.097601,POINT (-73.67037 41.09760)
2,09001010101h10,-73.667156,41.046854,POINT (-73.66716 41.04685),129288,129288,09,-73.667526,41.047768,POINT (-73.66753 41.04777)
3,09001010101h100,-73.720042,41.099381,POINT (-73.72004 41.09938),132056,132056,09,-73.720185,41.099778,POINT (-73.72019 41.09978)
4,09001010101h1000,-73.684385,41.056047,POINT (-73.68439 41.05605),236212,236212,09,-73.684882,41.055706,POINT (-73.68488 41.05571)
5,09001010101h1002,-73.684679,41.055643,POINT (-73.68468 41.05564),236212,236212,09,-73.684882,41.055706,POINT (-73.68488 41.05571)
6,09001010101h1001,-73.688061,41.090949,POINT (-73.68806 41.09095),557005,557005,09,-73.687929,41.090138,POINT (-73.68793 41.09014)
7,09001010101h1003,-73.689619,41.056749,POINT (-73.68962 41.05675),3275,3275,09,-73.690297,41.056475,POINT (-73.69030 41.05648)
8,09001010101h1004,-73.659922,41.048102,POINT (-73.65992 41.04810),236166,236166,09,-73.660382,41.047974,POINT (-73.66038 41.04797)
9,09001010101h1005,-73.667787,41.037095,POINT (-73.66779 41.03709),558618,558618,09,-73.667687,41.037297,POINT (-73.66769 41.03730)


### Apply Full Function

In [ ]:
#Apply 
if __name__=='__main__':
    print('Start running...')
    #set timer
    start_time = timeit.default_timer()
    #apply
    #wrk_r = parallelize(wrk, go)
    #pa_r = parallelize(homepa, go)
    ct_r = parallelize(homect, go)
    
    elapsed = timeit.default_timer() - start_time
    print("Total Time(s):", elapsed)
    print('End program')

Start running...


In [27]:
ct_r.head()

,hhold,long,lat,geometry,RID
0,42017100406h0,-74.855637,40.167156,POINT (-74.85564 40.16716),740321
1,42017100406h1,-74.846031,40.173643,POINT (-74.84603 40.17364),547316
2,42017100406h10,-74.844242,40.168111,POINT (-74.84424 40.16811),127096
3,42017100406h100,-74.846836,40.167468,POINT (-74.84684 40.16747),548459
4,42017100406h1000,-74.854315,40.155588,POINT (-74.85432 40.15559),550623


In [30]:
ct_r.iloc[:,[0,-1]].to_csv('home_ct_rid_oct.csv')